In [1]:
# From: https://www.kaggle.com/c/dog-breed-identification/data
# Author: Morpheus Hsieh

from __future__ import print_function, division

import os, sys
import copy
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir
from os.path import join, isfile
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import datasets, models, transforms, utils

In [2]:
ProcPath = r'D:\GitWork\dog_breed\data\processed'
print('Proc path:', ProcPath)

TestPath = r'D:\GitWork\dog_breed\data\raw\test'
print("Test path: '{}'".format(TestPath))

ModelPath = r'D:\GitWork\dog_breed\models'
print("Model path: '{}'".format(ModelPath))

BestModel = '20200916-160223_resnet50_acc94.pth'

BatchSize = 6
NUM_BREED_CLASSES = 16

CsvBreedsProc = 'breeds_processed.csv'
PreTrainedModel = 'resnet50-19c8e357.pth'


Proc path: D:\GitWork\dog_breed\data\processed
Test path: 'D:\GitWork\dog_breed\data\raw\test'
Model path: 'D:\GitWork\dog_breed\models'


In [3]:
# Read breed information from csv

f_abspath = join(ProcPath, CsvBreedsProc)
df_breeds = pd.read_csv(f_abspath)

print(df_breeds.info())
print(); print(df_breeds.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   breed_id  120 non-null    int64 
 1   breed     120 non-null    object
 2   count     120 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.9+ KB
None

   breed_id                 breed  count
0         0    scottish_deerhound    126
1         1           maltese_dog    117
2         2          afghan_hound    116
3         3           entlebucher    115
4         4  bernese_mountain_dog    114


In [4]:
NumClasses = 16
df_breeds_selected = df_breeds.head(NumClasses)

def df2dict(df, dir='fw'):
    dic = {}
    for i, row in df.iterrows():
        if dir == 'fw':   # fw = forward
            dic[row['breed']] = row['breed_id']
        elif dir == 'bw': # bw = backward
            key = str(row['breed_id'])
            dic[key] = row['breed']
    return dic

dict_breed_fw = df2dict(df_breeds_selected)
dict_breed_bw = df2dict(df_breeds_selected, 'bw')

print('Breeds dict forward:'); 
print(json.dumps(dict_breed_fw, indent=2))

Breeds dict forward:
{
  "scottish_deerhound": 0,
  "maltese_dog": 1,
  "afghan_hound": 2,
  "entlebucher": 3,
  "bernese_mountain_dog": 4,
  "shih-tzu": 5,
  "great_pyrenees": 6,
  "pomeranian": 7,
  "basenji": 8,
  "samoyed": 9,
  "airedale": 10,
  "tibetan_terrier": 11,
  "leonberg": 12,
  "cairn": 13,
  "beagle": 14,
  "japanese_spaniel": 15
}


In [5]:
# Dataset

normalize = transforms.Normalize(
    mean = [0.485, 0.456, 0.406],
    std  = [0.229, 0.224, 0.225]
)

# Transform
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])

class myDataset(Dataset):

    def __init__(self, path, transform=None):
        self.ext = '.jpg'
        ids = [
            f.replace(self.ext, '') \
            for f in listdir(path) if f.endswith(self.ext)
        ]
        ids.sort()
        
        self.path = path
        self.len = len(ids)
        self.img_id = ids
        self.labels = [0] * self.len
        self.transform = transform

    def __getitem__(self, index):
        img_id = self.img_id[index]
        f_abspath = join(self.path, img_id+self.ext)
        img_pil = Image.open(f_abspath)

        if self.transform is not None:
            img = self.transform(img_pil)

        lbl = int(self.labels[index])
        
        return [img, lbl]

    def __len__(self):
        return self.len
    
    def getID(self, index):
        return self.img_id[index]
    
dataset = myDataset(TestPath, transform=transform)
dataLoader = DataLoader(dataset, batch_size=BatchSize, shuffle=False)
datasize = len(dataLoader)

In [6]:
imgs, lbls = next(iter(dataLoader))
print('\nImage type:', type(imgs))
print('      size: ', imgs.size())

print('\nLabel type:', type(lbls))
print('      size: ', lbls.size())

img = imgs[0]
print('\nImage shape:', img.shape)
print(); print(img)

print('\nLabels:', lbls)


Image type: <class 'torch.Tensor'>
      size:  torch.Size([6, 3, 224, 224])

Label type: <class 'torch.Tensor'>
      size:  torch.Size([6])

Image shape: torch.Size([3, 224, 224])

tensor([[[-0.3712, -0.2856, -0.3541,  ..., -0.4397, -0.4226, -0.6794],
         [-0.4568, -0.2684, -0.5767,  ..., -0.5082, -0.6281, -0.7479],
         [-0.5424, -0.5596, -0.6965,  ..., -0.7822, -0.4054, -0.4739],
         ...,
         [-1.4843, -0.9534,  0.0912,  ..., -1.7583, -1.7754, -1.4672],
         [-1.8782, -1.5185, -1.0733,  ..., -1.6898, -1.5357, -1.4158],
         [-1.6898, -1.7412, -1.5185,  ..., -1.6042, -1.7925, -1.6384]],

        [[-0.0924, -0.1450, -0.3550,  ..., -0.1099, -0.1450, -0.4076],
         [-0.1800, -0.1099, -0.5476,  ..., -0.1625, -0.2850, -0.3901],
         [-0.3025, -0.4076, -0.6352,  ..., -0.3725,  0.0301, -0.0224],
         ...,
         [-1.1604, -0.8627,  0.2052,  ..., -1.2829, -1.2829, -0.9503],
         [-1.3179, -1.1604, -0.7227,  ..., -1.1954, -1.0203, -0.8978],
     

In [7]:
# prediction
use_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if use_gpu else "cpu")
print(device)

resnet = models.resnet50(pretrained=True)

model_abspath = join(ModelPath, PreTrainedModel)
resnet.load_state_dict(torch.load(model_abspath))

# for param in resnet.parameters():
#     param.requires_grad = False

num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, NUM_BREED_CLASSES)

resnet = resnet.cuda()

df = pd.DataFrame(columns=['id', 'breed'])

datapath = TestPath
id_list = [
    f.replace('.jpg', '') for f in listdir(datapath) \
    if f.endswith('.jpg') and isfile(join(datapath, f))
]

num = 10
for i in range(len(id_list[:num])):
    img_id = id_list[i]
    img_path = join(TestPath, img_id+'.jpg')
    img_pil = Image.open(img_path)
    img_tensor = transform(img_pil).unsqueeze_(0)
    inp = Variable(img_tensor)
    inp = inp.to(device)
    res = resnet(inp)
    index = res.data.cpu().numpy().argmax()
   
    df.loc[len(df)] = [img_id, index]


print()
print(df.info())
print()
print(df.head(num))


cuda:0

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10 non-null     object
 1   breed   10 non-null     object
dtypes: object(2)
memory usage: 240.0+ bytes
None

                                 id breed
0  000621fb3cbb32d8935728e48679680e    10
1  00102ee9d8eb90812350685311fe5890    10
2  0012a730dfa437f5f3613fb75efcd4ce    10
3  001510bc8570bbeee98c8d80c8a95ec1    10
4  001a5f3114548acdefa3d4da05474c2e    10
5  00225dcd3e4d2410dd53239f95c0352f    10
6  002c2a3117c2193b4d26400ce431eebd    10
7  002c58d413a521ae8d1a5daeb35fc803    10
8  002f80396f1e3db687c5932d7978b196    10
9  0036c6bcec6031be9e62a257b1c3c442    10
